# Corridor Comparison: I-680

In [1]:
import pandas as pd
from siuba import *
from calitp_data_analysis.sql import to_snakecase

### Importing the data
For this analysis, we need to read in data from:
* Caltrans Traffic Census Program: Using 2022 AADT Data
* Streetlight: Using Segment Analysis, Network Performance, and AADT for 2022
* Replica: Using AADT Count Application

In [2]:
ct_aadt = to_snakecase(pd.read_excel(
    "https://dot.ca.gov/-/media/dot-media/programs/traffic-operations/documents/census/2022/2022-traffic-volumes-ca.xlsx",
    sheet_name = "2022 AADT DATA"
    ))

In [3]:
ct_aadt.sample()

,district,route,route_sfx,county,pm_pfx,pm,pm_sfx,location_description,back_peak_hour,back_peak_madt,back_aadt,ahead_peak_hour,ahead_peak_madt,ahead_aadt
3514,8,86,NaN,RIV,R,22.157,NaN,DILLON RD,2550.0,34000.0,30200.0,2900.0,38500.0,34300.0


In [4]:
## Streetlight AADT
sl_aadt = to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/big_data/i680/1751074_I680_AADT_Test_estimated_aadt.csv"))

In [5]:
## Streetlight Network Performance Test
sl_npt = to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/big_data/i680/1751071_I680_network_performance_test_network_performance.csv"))

In [6]:
## Streetlight Segment Analysis
sl_sa = to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/big_data/i680/1751075_i680_Segment_Analysis_test_sa_all.csv"))

In [7]:
## Joining the streetlight segments so that all STreetlight Analyses are together

In [8]:
sl_aadt = (sl_aadt>>select(_.data_periods, _.zone_name,
                           _.estimated_2022_aadt, _.lower_95_percent_prediction_interval,
                           _.upper_95_percent_prediction_interval))

In [9]:
sl_aadt.columns = ['aadt_' + str(col) for col in sl_aadt.columns]


In [18]:
sl_aadt = sl_aadt.rename(columns={"aadt_zone_name": "zone_name"}) 

In [11]:
sl_npt = (sl_npt>>filter(_.day_part=="0: All Day (12am-12am)")>>select(_.data_periods, _.zone_name,
                                            _.average_daily_segment_traffic__stl_volume_))

In [12]:
sl_npt.columns = ['npt_' + str(col) for col in sl_npt.columns]

In [19]:
sl_npt = sl_npt.rename(columns={"npt_zone_name": "zone_name"}) 

In [14]:
sl_sa = ((sl_sa>>filter(_.day_part=="0: All Day (12am-12am)")>>select(_.data_periods, 
                                                                    _.zone_name,
                                                                     _.average_daily_segment_traffic__stl_volume_)))

In [15]:
sl_sa.columns = ['sa_' + str(col) for col in sl_sa.columns]

In [20]:
sl_sa = sl_sa.rename(columns={"sa_zone_name": "zone_name"}) 

In [22]:
sl = pd.merge(sl_aadt, sl_npt, on="zone_name")


In [25]:
sl = pd.merge(sl, sl_sa, on = "zone_name")

In [26]:
sl

,aadt_data_periods,zone_name,aadt_estimated_2022_aadt,aadt_lower_95_percent_prediction_interval,aadt_upper_95_percent_prediction_interval,npt_data_periods,npt_average_daily_segment_traffic__stl_volume_,sa_data_periods,sa_average_daily_segment_traffic__stl_volume_
0,"Jan 01, 2022 - Dec 31, 2022",Benicia-Martinez Bridge / 123867334 / 1,39493,31800,48800,"Oct 01, 2021 - Sep 30, 2022",35593,"Oct 01, 2021 - Apr 30, 2022",38336
1,"Jan 01, 2022 - Dec 31, 2022",Benicia-Martinez Bridge / 439915393 / 1,38653,31100,47800,"Oct 01, 2021 - Sep 30, 2022",35465,"Oct 01, 2021 - Apr 30, 2022",38092
2,"Jan 01, 2022 - Dec 31, 2022",Benicia-Martinez Bridge / 4968908 / 1,57224,46500,69000,"Oct 01, 2021 - Sep 30, 2022",57498,"Oct 01, 2021 - Apr 30, 2022",53603
3,"Jan 01, 2022 - Dec 31, 2022",Benicia-Martinez Bridge / 52000448 / 1,60529,49200,73000,"Oct 01, 2021 - Sep 30, 2022",61440,"Oct 01, 2021 - Apr 30, 2022",55628
4,"Jan 01, 2022 - Dec 31, 2022",Benicia-Martinez Bridge / 8703049 / 1,38663,31100,47800,"Oct 01, 2021 - Sep 30, 2022",35173,"Oct 01, 2021 - Apr 30, 2022",38094
...,...,...,...,...,...,...,...,...,...
556,"Jan 01, 2022 - Dec 31, 2022",Sinclair Freeway / 961140461 / 1,84762,70000,99800,"Oct 01, 2021 - Sep 30, 2022",88248,"Oct 01, 2021 - Apr 30, 2022",99848
557,"Jan 01, 2022 - Dec 31, 2022",Sinclair Freeway / 961140462 / 1,88021,72700,103600,"Oct 01, 2021 - Sep 30, 2022",95175,"Oct 01, 2021 - Apr 30, 2022",104091
558,"Jan 01, 2022 - Dec 31, 2022",Sinclair Freeway / 961140463 / 1,87816,72500,103400,"Oct 01, 2021 - Sep 30, 2022",80341,"Oct 01, 2021 - Apr 30, 2022",102763
559,"Jan 01, 2022 - Dec 31, 2022",Sinclair Freeway / 986291461 / 1,57891,47000,69800,"Oct 01, 2021 - Sep 30, 2022",48800,"Oct 01, 2021 - Apr 30, 2022",53177
